In [1]:
import numpy as np
from astropy.table import Table, vstack, hstack

In [2]:
#Read the full log. 
log = Table.read("../../proc/log.fits")

In [3]:
#Separate between Hot DOGs and calibrators, as we want first all Hot DOGs and then the calibrators.
cond = log['Program_ID']=="60.A-9203(E)"
hds = log[~cond]
cals = log[cond]

In [4]:
#Put the target names in order. First Hot DOGs, then Cal Stars and then Zero Pol stars. As all zero pol stars have names starting WD, cals is already arange in the proper way. 
targs = np.concatenate([np.unique(hds['Target'].astype(str)), np.unique(cals['Target'].astype(str))])

In [5]:
#Main cycle.
for i, targ in enumerate(targs):
    for j, filt in enumerate(np.unique(log['Filter'][log['Target']==targ])):
        pcat_name = "../../analysis/{}_pol_{}.dat".format(targ,filt)
        pcat = Table.read(pcat_name,format='ascii')
        #Make sure the first column is a string
        if pcat['col1'].dtype == "int64":
            pcat['col1'] = pcat['col1'].astype(str)
        #Not all tables have col6 (mean MJD)
        if 'col6' in pcat.colnames:
            pcat.remove_column('col6')
        #Add the WISE ID and filter name. 
        pcat = hstack([[targ]*len(pcat), [filt]*len(pcat), pcat])
        #If first iteration, create the table.
        if i==0 and j==0:
            final_table = pcat
        else:
            #Concatenate.
            final_table = vstack([final_table, pcat])

In [6]:
#Rename the columns.
colids = ["Target", "Band", "OB", "pol", "dpol", "chi", "dchi"]
for j, col in enumerate(final_table.colnames):
    final_table.rename_column(col, colids[j])

In [7]:
#Transform degrees of polarization into percentages.
final_table['pol'] *= 100.
final_table['dpol'] *= 100.

In [8]:
#We will create two ne columns that show measurement and error for pol frac and angle. 
final_table['P'] = np.zeros(len(final_table), dtype='<U18')
final_table['\chi'] = np.zeros(len(final_table), '<U18')
for i in range(len(final_table)):
    final_table['P'][i] = "{:.2f}$\\pm${:.2f}".format(final_table['pol'][i], final_table['dpol'][i])
    final_table['\chi'][i] = "{:.1f}$\\pm${:.1f}".format(final_table['chi'][i], final_table['dchi'][i])

In [9]:
#Remove unneeded columns.
for col in ['pol', 'dpol', 'chi', 'dchi']:
    final_table.remove_column(col)

In [10]:
#Save the table as latex.
final_table.write("pol_table_aux.tex", format='ascii.latex', overwrite=True, formats={'Target':'%020s'})

In [11]:
#Save the table as latex.
final_table.write("pol_table_aux.txt", format='ascii.fixed_width', overwrite=True)